<div style="background-color:rgb(0, 55, 207); padding: 30px; border-radius: 20px; box-shadow: 0 4px 15px rgba(105, 195, 255, 0.3); color:rgb(187, 201, 248); font-family: 'Times New Roman', serif;">

<h1 style="text-align: center; font-size: 38px; color: white; font-weight: bold;">Creating Numpy Files of Extracted Landmarks</h1>

<h3 style="font-size: 22px; color: white; font-weight: bold;">Libraries</h3>

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from pathlib import Path
from tqdm import tqdm

<h3 style="font-size: 22px; color: white; font-weight: bold;">Files Processing</h3>

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

DATA_DIR = Path(r"YOUR_DATASET")
OUTPUT_DIR = Path(r"OUTPUT")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def extract_hand_keypoints(results):
    # Get left and right hand landmarks
    lh = np.array([[res.x, res.y, res.z] 
                   for res in results.multi_hand_landmarks[0].landmark]) if results.multi_hand_landmarks and len(results.multi_hand_landmarks) > 0 else np.zeros(21*3)
    
    rh = np.array([[res.x, res.y, res.z] 
                   for res in results.multi_hand_landmarks[1].landmark]) if results.multi_hand_landmarks and len(results.multi_hand_landmarks) > 1 else np.zeros(21*3)
    
    return np.concatenate([lh, rh])

def process_video(video_path):
    cap = cv2.VideoCapture(str(video_path))
    landmarks_seq = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            keypoints = extract_hand_keypoints(results)
            landmarks_seq.append(keypoints)

    cap.release()
    return np.array(landmarks_seq, dtype=np.float32)

# Process each video
for video_file in tqdm(list(DATA_DIR.rglob("*.mp4"))):
    relative_path = video_file.relative_to(DATA_DIR)
    output_path = OUTPUT_DIR / relative_path.with_suffix(".npy")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    landmarks_array = process_video(video_file)
    np.save(output_path, landmarks_array)
